# Apify Actor

>[Apify Actors](https://docs.apify.com/platform/actors) 是为各种网络抓取、爬取和数据提取任务设计的云程序。这些 actor 促进了从网络自动收集数据，使用户能够高效地提取、处理和存储信息。Actor 可用于执行诸如抓取电商网站的产品详情、监控价格变动或收集搜索引擎结果之类的任务。它们与 [Apify Datasets](https://docs.apify.com/platform/storage/dataset) 无缝集成，使得 actor 收集的结构化数据能够以 JSON、CSV 或 Excel 等格式存储、管理和导出，以供进一步分析或使用。

## 概览

本 Notebook 将引导您了解如何将 [Apify Actors](https://docs.apify.com/platform/actors) 与 LangChain 结合使用，以自动化网络抓取和数据提取。`langchain-apify` 包将 Apify 的云工具与 LangChain Agents 集成，为 AI 应用程序实现高效的数据收集和处理。

## 设置

此集成位于 `langchain-apify` 包中。可以使用 pip 来安装该包。

In [ ]:
%pip install langchain-apify

### 先决条件

- **Apify 账户**: 在 [此处](https://console.apify.com/sign-up) 注册您的免费 Apify 账户。
- **Apify API 令牌**: 在 [Apify 文档](https://docs.apify.com/platform/integrations/api) 中了解如何获取您的 API 令牌。

In [ ]:
import os

os.environ["APIFY_API_TOKEN"] = "your-apify-api-token"
os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

## 实例化

我们实例化 `ApifyActorsTool` 以便调用 [RAG Web Browser](https://apify.com/apify/rag-web-browser) Apify Actor。此 Actor 为 AI 和 LLM 应用程序提供网页浏览功能，类似于 ChatGPT 中的网页浏览功能。[Apify Store](https://apify.com/store) 中的任何 Actor 都可以这样使用。

In [ ]:
from langchain_apify import ApifyActorsTool

tool = ApifyActorsTool("apify/rag-web-browser")

## 调用

`ApifyActorsTool` 接受一个参数，即 `run_input` - 一个字典，它将被传递给 Actor 作为运行输入。运行输入模式文档可以在 Actor 详情页的输入部分找到。请参阅 [RAG Web Browser 输入模式](https://apify.com/apify/rag-web-browser/input-schema)。

In [ ]:
tool.invoke({"run_input": {"query": "what is apify?", "maxResults": 2}})

## 链式调用

我们可以将创建的工具提供给一个[代理（agent）](https://python.langchain.com/docs/tutorials/agents/)。当代理被要求搜索信息时，它将调用 Apify Actor，该 Actor 会搜索网络，然后检索搜索结果。

In [ ]:
%pip install langgraph langchain-openai

In [ ]:
from langchain_core.messages import ToolMessage
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

model = ChatOpenAI(model="gpt-4o")
tools = [tool]
graph = create_react_agent(model, tools=tools)

In [ ]:
inputs = {"messages": [("user", "search for what is Apify")]}
for s in graph.stream(inputs, stream_mode="values"):
    message = s["messages"][-1]
    # skip tool messages
    if isinstance(message, ToolMessage):
        continue
    message.pretty_print()

================================ Human Message =================================

search for what is Apify
================================== Ai Message ==================================
Tool Calls:
  apify_actor_apify_rag-web-browser (call_27mjHLzDzwa5ZaHWCMH510lm)
 Call ID: call_27mjHLzDzwa5ZaHWCMH510lm
  Args:
    run_input: {"run_input":{"query":"Apify","maxResults":3,"outputFormats":["markdown"]}}
================================== Ai Message ==================================

Apify is a comprehensive platform for web scraping, browser automation, and data extraction. It offers a wide array of tools and services that cater to developers and businesses looking to extract data from websites efficiently and effectively. Here's an overview of Apify:

1. **Ecosystem and Tools**:
   - Apify provides an ecosystem where developers can build, deploy, and publish data extraction and web automation tools called Actors.
   - The platform supports various use cases such as extracting data fr

## API 参考

有关如何使用此集成的更多信息，请参阅 [git 存储库](https://github.com/apify/langchain-apify) 或 [Apify 集成文档](https://docs.apify.com/platform/integrations/langgraph)。